In [2]:
import json
import time
import random

from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka.admin import AdminClient
from confluent_kafka import Producer, Consumer, KafkaError, TopicPartition

# Create Topic

In [3]:
admin_client = AdminClient({'bootstrap.servers': 'broker:9092'})

# Retrieve the list of topics
topic_metadata = admin_client.list_topics(timeout=10)
[topic for topic in topic_metadata.topics if not topic.startswith('_')]

[]

In [4]:
# create new topic
new_topics = [NewTopic('trades', num_partitions=3, replication_factor=1)]
fs = admin_client.create_topics(new_topics)
for topic, f in fs.items():
    try:
        f.result()
        print("Topic {} created".format(topic))
    except Exception as e:
        print("Failed to create topic {}: {}".format(topic, e))

Topic trades created


In [5]:
topics = admin_client.list_topics().topics
for topic, topic_info in topics.items():
    if not topic.startswith('_'):
        print(f"Topic: {topic}, Partitions: {len(topic_info.partitions)}")
        for p_id, p_info in topic_info.partitions.items():
            print(f"  Partition: {p_id}, Leader: {p_info.leader}, Replicas: {p_info.replicas}")

Topic: trades, Partitions: 3
  Partition: 0, Leader: 1, Replicas: [1]
  Partition: 1, Leader: 1, Replicas: [1]
  Partition: 2, Leader: 1, Replicas: [1]


# Producer

In [6]:
trades = [
    {
        'e': 'trade',
        'E': 1713991342890,
        's': 'BTCUSDT',
        't': 97236,
        'p': '0.006390',
        'q': '2551',
        'b': 69,
        'a': 607,
        'T': 1713991342890,
        'm': True,
        'M': True
    },
    {
        'e': 'trade',
        'E': 1713991378161,
        's': 'BNBUSDT',
        't': 77422,
        'p': '0.008396',
        'q': '2831',
        'b': 307,
        'a': 539,
        'T': 1713991378161,
        'm': False,
        'M': True
    },
    {
        'e': 'trade',
        'E': 1713991429451,
        's': 'BNBBTC',
        't': 43014,
        'p': '0.004122',
        'q': '3493',
        'b': 123,
        'a': 664,
        'T': 1713991429451,
        'm': True,
        'M': True
    },
]

In [7]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print(f"Message produced: {msg.topic()} {msg.partition()} {msg.key()}")

producer = Producer({'bootstrap.servers': 'broker:9092'})

for trade in trades:
    producer.produce('trades', key=trade['s'], value=json.dumps(trade), callback=acked)

# Wait for any outstanding messages to be delivered
producer.flush()

Message produced: trades 2 b'BNBUSDT'
Message produced: trades 0 b'BTCUSDT'
Message produced: trades 1 b'BNBBTC'


0

# Consumer

In [10]:
from confluent_kafka import Consumer, KafkaError, TopicPartition
import time

# Configure the consumer
consumer_config = {
    'bootstrap.servers': 'broker:9092',
    'group.id': 'group_1',
    'auto.offset.reset': 'earliest'
}

# Initialize the consumer
consumer = Consumer(consumer_config)

# Subscribe to the topic
topic_name = 'trades'
consumer.subscribe([topic_name])


try:
    start_time = time.time()
    while True:
        if time.time() - start_time > 10:  # Break the loop after 10 seconds
            break
            
        msg = consumer.poll(timeout=1.0)  # Poll for a message (timeout in seconds)
        if msg is None:
            continue  # No message available within the timeout period
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                print(f'Reached end of {msg.partition()} at offset {msg.offset()}')
            else:
                print(f'Error: {msg.error()}')
            continue

        # Access and print message details
        print(f'Message: {msg.value().decode("utf-8")}')
        print(f'Partition: {msg.partition()}')
        print(f'Offset: {msg.offset()}')
finally:
    consumer.close()

Message: {"e": "trade", "E": 1713991378161, "s": "BNBUSDT", "t": 77422, "p": "0.008396", "q": "2831", "b": 307, "a": 539, "T": 1713991378161, "m": false, "M": true}
Partition: 2
Offset: 0
Message: {"e": "trade", "E": 1713991429451, "s": "BNBBTC", "t": 43014, "p": "0.004122", "q": "3493", "b": 123, "a": 664, "T": 1713991429451, "m": true, "M": true}
Partition: 1
Offset: 0
Message: {"e": "trade", "E": 1713991342890, "s": "BTCUSDT", "t": 97236, "p": "0.006390", "q": "2551", "b": 69, "a": 607, "T": 1713991342890, "m": true, "M": true}
Partition: 0
Offset: 0
